# 酵母画像特徴量データから、細胞状態の予測

## データの読み込みと標準化

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

# データの読み込み
data = pd.read_csv("data/yeast_his3.csv")
columns = ["C101", "C103", "C104","C115","A101","A120","A121","A122","A123"]
cell_features_pre = data[["Cgroup"] + columns]
cell_features = cell_features_pre[np.sum(cell_features_pre.isnull(), axis=1) == 0]
# 標準化
X = cell_features[columns] 
X_norm = preprocessing.StandardScaler().fit_transform(X)

今回は芽の大きさ("Cgroup")の予測を実施しますが、現在"Cgroup" の値は、"large", "medium", "small", "no"の4種類になっています。一方で、機械学習ライブラリが扱える値は、数値(0,1,2,3)なので、文字列→数値の変換を実施します。

In [3]:
y = np.zeros(X.shape[0])
y[cell_features["Cgroup"] == "medium"] = 1
y[cell_features["Cgroup"] == "small"] = 2
y[cell_features["Cgroup"] == "no"] = 3

変換前後で、各クラスの数値が異なっていないか確認を行います。

In [4]:
import collections
collections.Counter(np.array(cell_features["Cgroup"]))

Counter({'medium': 88, 'no': 114, 'large': 97, 'small': 73})

In [5]:
collections.Counter(y)

Counter({1.0: 88, 3.0: 114, 0.0: 97, 2.0: 73})

## データの分割

機械学習では、データを２分割し、片方でモデルの学習を実施、もう一方で予測精度の検証を行います。
ここでは、全体の2/3の細胞を利用して学習を実施し(training data)、残りの1/3の細胞で予測精度の検証を行います(test data)。

データ分割は、scikit learn に実装されているため、以下のコマンド一つで実施できます。

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.33)

分割されたデータ数を確認します。

In [7]:
print("X: %s\tX_train: %s\tX_test: %s" % (X_norm.shape, X_train.shape, X_test.shape))
print("y: %d\ty_train:%d\ty_test:%d" % (len(y), len(y_train), len(y_test)))

X: (372, 9)	X_train: (249, 9)	X_test: (123, 9)
y: 372	y_train:249	y_test:123


In [8]:
collections.Counter(y_train)

Counter({2.0: 51, 1.0: 54, 3.0: 80, 0.0: 64})

In [9]:
collections.Counter(y_test)

Counter({2.0: 22, 0.0: 33, 1.0: 34, 3.0: 34})

## SVMの実施

SVMの実装の内、クラス分類問題向けに作られている関数 SVCで、SVMを実行します。

本来であれば、カーネル関数を何にするか、SVMの式中に現れるペナルティCをいくつに設定するかなどのハイパーパラメータを最適化する必要があるが、ここでは全体の手順のみに注力するため、ハイパーパラメータの最適化は実施しません。

In [10]:
from sklearn.svm import SVC
clf = SVC(gamma='auto') #　実行するクラス分類機の設定。ここではSVM
clf.fit(X_train, y_train) # クラス分類の実行、モデルの作成

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

作成したモデルを基に、テストデータ上での正答率を計測します。ここでは、確認のため、同時に訓練データでの正答率も確認します。
訓練ーテストデータの分割での、学習時にも乱数を利用しているため、皆さんのPCで実行した場合に、正答率が異なる可能性があります。

In [11]:
# 訓練データでの精度（参考）
y_train_pred = clf.predict(X_train)
print("Training accuracy: ", sum(y_train_pred == y_train) / len(y_train))
# テストデータでの精度（本番）
y_test_pred = clf.predict(X_test)
print("Test accuracy: ", sum(y_test_pred == y_test) / len(y_test))

Training accuracy:  0.8835341365461847
Test accuracy:  0.7317073170731707


Accuracyは、正誤の判定だけだが、実際には "large"を"no"に間違えるのか、"large"を"medium"に間違えるのかで、意味合いが異なる可能性があります。どのように間違えているのかを、分割表を書くことで確認します。

pandasに変換しているのは、表をキレイに表示するためだけで、本質的な意味はありません。

In [12]:
from sklearn.metrics import confusion_matrix
targets = ['large', 'medium', 'small', 'no']
pd.DataFrame(confusion_matrix(y_train, y_train_pred), columns=targets) # Training

,large,medium,small,no
0,63,1,0,0
1,8,40,2,4
2,0,2,38,11
3,0,0,1,79


In [13]:
pd.DataFrame(confusion_matrix(y_test, y_test_pred), columns=targets) # Test

,large,medium,small,no
0,30,3,0,0
1,8,21,5,0
2,0,2,9,11
3,0,0,4,30


この分割表から、間違えていても多くは近い値であって、"large" を "no" に間違えるようなことは、起こっていないと分かります。

## Random Forest の実施

SVM同様に、Random Forest を実施する。変更点は、SVCがRandomForestClassifier になるだけ。

In [14]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(X_norm,y)
# 訓練データでの精度（参考）
y_train_pred = clf.predict(X_train)
print("Training accuracy: ", sum(y_train_pred == y_train) / len(y_train))
# テストデータでの精度（本番）
y_test_pred = clf.predict(X_test)
print("Test accuracy: ", sum(y_test_pred == y_test) / len(y_test))

Training accuracy:  0.8995983935742972
Test accuracy:  0.9105691056910569


SVMに比べて、高い精度を出しているように見えます。分割表を書いてみましょう。

In [15]:
pd.DataFrame(confusion_matrix(y_train, y_train_pred), columns=targets) # Training

,large,medium,small,no
0,59,5,0,0
1,6,46,1,1
2,0,5,44,2
3,0,0,5,75


In [16]:
pd.DataFrame(confusion_matrix(y_test, y_test_pred), columns=targets) # Test

,large,medium,small,no
0,30,3,0,0
1,0,33,1,0
2,0,2,17,3
3,0,0,2,32


SVM同様に大きな誤りは少なく、高い精度で予測できていることがわかります。

以上で、SVMとRandomForestを利用して、酵母の画像から得られた特徴量を利用して、細胞周期（芽の大きさ）が、ある程度良く可能であることがわかりました。次は、画像自身から特徴量の陽な抽出を無く、細胞周期の予測を行います。